# Plot model patterns for a schematic

In [1]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import FastICA
from mindstorm import dsmplot
from cymr import cmr

res_dir = Path(os.environ['CFR_RESULTS'])
fig_dir = Path(os.environ['CFR_FIGURES']) / 'schematic'

In [2]:
patterns = cmr.load_patterns(res_dir / 'cfr_patterns.hdf5')
c_init = np.tile(np.sqrt(1 / 9), 9)

In [3]:
items = [
    'Mount Fuji', 
    'Oprah Winfrey', 
    'cup', 
    'Jennifer Aniston', 
    'Mount Everest', 
    'backpack', 
    'Eiffel Tower',
    'White House',
    'Wall Street',
    'birdhouse',
    'speaker',
    'orange',
    'Mike Myers',
    'Abraham Lincoln',
]
categories = [1, 0, 2, 0, 1, 2, 1, 1, 1, 2, 2, 0, 0]
ind = [np.where(patterns['items'] == i.upper())[0][0] for i in items]

In [4]:
n = len(items)
m = 9
ica = FastICA(n_components=m, random_state=42)
raw_vectors = patterns['vector']['use'][ind, :]
vectors = np.clip(stats.zscore(ica.fit_transform(raw_vectors), axis=0), -1, 1)

In [5]:
def update(c, c_in, B):
    cdot = np.dot(c, c_in)
    rho = np.sqrt(1 + B ** 2 * (cdot ** 2 - 1)) - (B * cdot)
    c_new = rho * c + B * c_in
    return c_new

In [6]:
def print_pattern(x, fig_file):
    mat = x.reshape(3, 3)
    fig = plt.figure(frameon=False, figsize=(2, 2))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    h = ax.matshow(mat, cmap="gray")
    h.set_clim(0, 1)
    plt.savefig(fig_file, pad_inches=0)
    plt.close(fig)

In [7]:
B1 = 1
B2 = 0.5
c1 = c_init.copy()
c2 = c_init.copy()
for item, c_in in zip(items, vectors):
    c1 = update(c1, c_in, B1)
    c2 = update(c2, c_in, B2)
    print_pattern(c1, fig_dir / f'use_input_{item}.png')
    print_pattern(c2, fig_dir / f'use_blend_{item}.png')

In [8]:
rng = np.random.default_rng(42)

In [9]:
B1 = 1
B2 = 0.8
c1 = c_init.copy()
c2 = c_init.copy()
indices = np.arange(9)
rng.shuffle(indices)
for item, c in zip(items, categories):
    pattern = np.zeros(9)
    pattern[indices[c * 3:(c + 1) * 3]] = 1
    c1 = update(c1, pattern, B1)
    c2 = update(c2, pattern, B2)
    print_pattern(c1, fig_dir / f'cat_input_{item}.png')
    print_pattern(c2, fig_dir / f'cat_blend_{item}.png')

In [10]:
B1 = 1
B2 = 0.7
c1 = c_init.copy()
c2 = c_init.copy()
indices = np.arange(9)
rng.shuffle(indices)
n = 9
for i, item in enumerate(items[:n]):
    pattern = np.zeros(n)
    pattern[indices[i]] = 1
    c1 = update(c1, pattern, B1)
    c2 = update(c2, pattern, B2)
    print_pattern(c1, fig_dir / f'item_input_{item}.png')
    print_pattern(c2, fig_dir / f'item_blend_{item}.png')

In [11]:
%load_ext watermark
%watermark -v -iv

Python implementation: CPython
Python version       : 3.10.13
IPython version      : 8.16.1

cymr      : 0.12.1
matplotlib: 3.8.0
numpy     : 1.26.1
scipy     : 1.11.3
mindstorm : 0.11.1

